In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import FixedPoint
from cicada import transcript

def main(communicator):
    handler = logging.FileHandler(f"player-{communicator.rank}.log", mode="w")
    handler.setFormatter(transcript.Formatter())
    handler.addFilter(transcript.ShowSentMessages())
    handler.addFilter(transcript.ShowReceivedMessages())
    handler.addFilter(transcript.HideSpecialFunctions())
    handler.addFilter(transcript.HideInitFunctions())
    handler.addFilter(transcript.HidePrivateFunctions())
    handler.addFilter(transcript.HideDefaultFunctions())
    handler.addFilter(transcript.HideCommunicationFunctions())
    handler.addFilter(transcript.HideSelfArguments())
    handler.addFilter(transcript.MapInlineResults({
        "cicada.arithmetic.Field.inplace_add": "lhs",
        "cicada.arithmetic.Field.inplace_subtract": "lhs",
    }))
    transcript.logger.addHandler(handler)

    transcript.log("Let's share some secrets!", comm=communicator)
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!", comm=communicator)
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!", comm=communicator)
    c = protocol.reveal(c_share)

    handler.close()

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3)

In [3]:
for rank in range(3):
    with open(f"player-{rank}.log", "r") as stream:
        print(stream.read())
        print("\n" * 5)

Player 0: Let's share some secrets!
Player 0: comm world player 0: --> 1 PRZS 1388654068352101360
Player 0: comm world player 0: <-- 2 PRZS 914449666196114367
Player 0: AdditiveProtocolSuite.share(src=0, secret=array(2), shape=(), encoding=None)
Player 0:   Field.uniform(size=(), generator=Generator(PCG64) at 0x111E252A0)
Player 0:     Field.uniform => 211
Player 0:   Field.uniform(size=(), generator=Generator(PCG64) at 0x111E25460)
Player 0:     Field.uniform => 105
Player 0:   Field.inplace_subtract(lhs=array(211, dtype=object), rhs=array(105, dtype=object))
Player 0:     Field.inplace_subtract => 106
Player 0:   FixedPoint.encode(array=array(2), field=cicada.arithmetic.Field(order=127))
Player 0:     FixedPoint.encode => 8
Player 0:   Field.inplace_add(lhs=array(106, dtype=object), rhs=array(8, dtype=object))
Player 0:     Field.inplace_add => 114
Player 0:   AdditiveProtocolSuite.share => cicada.additive.AdditiveArrayShare(storage=114)
Player 0: AdditiveProtocolSuite.share(src=1, s